In [ ]:
import cv2
import numpy as np

# Load video clip
cap = cv2.VideoCapture('D:/video.mp4')
bg_image = cv2.imread("D:/background.png")

# Get the height and width of the frame
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
bg_image = cv2.resize(bg_image, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    background = bg_image.copy()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define two ranges for red in HSV (red is split at 0° and 180° hue)
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    # Create masks and combine them
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Improve mask quality
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.dilate(mask, kernel)
    mask_inv = cv2.bitwise_not(mask)

    removed_bg = cv2.bitwise_and(frame, frame, mask=mask_inv)
    background = cv2.bitwise_and(background, background, mask=mask)

    final = cv2.add(removed_bg, background)
    cv2.imshow("result", final)
    cv2.waitKey(25)

cap.release()
cv2.destroyAllWindows()
